# Database Setup Notebook

This notebook builds the entire MySQL database for the Big Data project.

The steps performed:

- [Step 0 — Import Libraries](#step20)
- [Step 1 — Define File Paths & Abbreviations](#step21)
- [Step 2 — Detect Encodings](#step22)
- [Step 3 — Clean CSV Files](#step23)
- [Step 4 — Convert to Long Format (`all_long`)](#step24)
- [Step 5 — Build `countries_df`](#step25)
- [Step 6 — Build `indicators_df`](#step26)
- [Step 7 — Build `values_df`](#step27)
- [Step 8 — Connect to MySQL & Create DB](#step28)
- [Step 9 — Reset Schema & Create Tables](#step29)
- [Step 10 — Insert Countries & Indicators](#step210)
- [Step 11 — Bulk Insert `indicator_values`](#step211)
- [Step 12 — Create SQL View `all_data`](#step212)
- [Step 13 — Sanity Checks](#step213)

## STEP 0 — Import all required libraries


In [1]:
import os
import math
import re

import numpy as np
import pandas as pd
import pymysql

import custom_functions  # cleaning & encoding helpers

pd.options.display.max_columns = 100


## STEP 1 — Define file paths and abbreviations

We list the 8 CSV files (health + environmental indicators) using the same paths as in `main.ipynb`.

They are split into:
- **Notation 1:** semicolon-delimited, clean files  
- **Notation 2:** comma-delimited, messy files requiring extra cleaning  


In [2]:
# File paths for notation 1 ('che', 'wr', 'wu', 'sr', 'su', 'gem')
filepaths_1 = [
    'che_cleaned.csv',
    'wr_cleaned.csv',
    'wu_cleaned.csv',
    'sr_cleaned.csv',
    'su_cleaned.csv',
    'gem_cleaned.csv'
]

# File paths for notation 2 ('pop', 'ren')
filepaths_2 = [
    'pop_cleaned.csv',
    'ren_cleaned.csv'
]

# Abbreviations
abbreviations_1 = ['che', 'wr', 'wu', 'sr', 'su', 'gem']
abbreviations_2 = ['pop', 'ren']


## STEP 2 — Detect encodings for all CSV files

We use `custom_functions.detect_encoding()` to ensure each file is read with the correct encoding.


In [3]:
encodings_1 = {}
for abbreviation, filepath in zip(abbreviations_1, filepaths_1):
    encodings_1[abbreviation] = custom_functions.detect_encoding(filepath)

print("Encodings for notation 1 files:\n", encodings_1)

encodings_2 = {}
for abbreviation, filepath in zip(abbreviations_2, filepaths_2):
    encodings_2[abbreviation] = custom_functions.detect_encoding(filepath)

print("Encodings for notation 2 files:\n", encodings_2)


Encodings for notation 1 files:
 {'che': 'ascii', 'wr': 'ascii', 'wu': 'ascii', 'sr': 'ascii', 'su': 'ascii', 'gem': 'ascii'}
Encodings for notation 2 files:
 {'pop': 'ascii', 'ren': 'ascii'}


## STEP 3 — Clean all CSV files using your custom functions

Notation 1 uses:
- `;` separator
- rows start at index 3

Notation 2 uses:
- `,` separator
- many quotes, commas, extra characters
- rows start at index 4


In [ ]:
# 3.1 Notation 1
dfnames_1 = ['df_che', 'df_wr', 'df_wu', 'df_sr', 'df_su', 'df_gem']
df_dict_1 = {}

for df_name, abbreviation, filepath in zip(dfnames_1, abbreviations_1, filepaths_1):
    df_dict_1[df_name] = custom_functions.clean_csv(
        filepath=filepath,
        encoding=encodings_1[abbreviation],
        separator=',',
        trail1='\n',
        trail2=None,
        trail3=None,
        to_be_replaced='"',
        start_row=3
    )

df_che = df_dict_1['df_che']
df_wr  = df_dict_1['df_wr']
df_wu  = df_dict_1['df_wu']
df_sr  = df_dict_1['df_sr']
df_su  = df_dict_1['df_su']
df_gem = df_dict_1['df_gem']

# 3.2 Notation 2
dfnames_2 = ['df_pop', 'df_ren']
df_dict_2 = {}

for df_name, abbreviation, filepath in zip(dfnames_2, abbreviations_2, filepaths_2):
    df_dict_2[df_name] = custom_functions.clean_csv(
        filepath=filepath,
        encoding=encodings_2[abbreviation],
        separator=',',
        trail1='\n',
        trail2='"',
        trail3=',',
        to_be_replaced='"',
        start_row=4
    )

df_pop = df_dict_2['df_pop']
df_ren = df_dict_2['df_ren']

print("Shapes of cleaned DataFrames:")
for name, df in {
    "che": df_che, "wr": df_wr, "wu": df_wu,
    "sr": df_sr, "su": df_su, "gem": df_gem,
    "pop": df_pop, "ren": df_ren
}.items():
    print(f"{name}: {df.shape}")


## STEP 4 — Convert all datasets into a unified long-format table

Each dataset is converted from wide (1960–2024 columns) into long format:


In [7]:
headers_1 = {
    'che': df_che.columns.tolist(),
    'wr': df_wr.columns.tolist(),
    'wu': df_wu.columns.tolist(),
    'sr': df_sr.columns.tolist(),
    'su': df_su.columns.tolist(),
    'gem': df_gem.columns.tolist()
}
headers_2 = {
    'pop': df_pop.columns.tolist(),
    'ren': df_ren.columns.tolist()
}
print("Headers of cleaned DataFrames:")
for name, headers in {**headers_1, **headers_2}.items():
    print(f"{name}: {headers}")
    

NameError: name 'df_che' is not defined

In [6]:

# Melting function- Long format

def melt_indicator(df):
    # 1. FORCE RENAME the first 4 columns to ensure they match id_vars
    # This fixes issues with capitalization ('country name') or typos
    df.columns.values[0] = "Country Name"
    df.columns.values[1] = "Country Code"
    df.columns.values[2] = "Indicator Name"
    df.columns.values[3] = "Indicator Code"

    # 2. Identify year columns (all columns except the first 4)
    # We use slicing [4:] instead of isdigit() to be safer against bad headers
    year_cols = df.columns[4:]

    # 3. Melt
    long_df = df.melt(
        id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
        value_vars=year_cols,
        var_name="Year",
        value_name="Value"
    )

    # 4. Clean up types
    long_df["Year"] = pd.to_numeric(long_df["Year"], errors="coerce")
    long_df["Value"] = pd.to_numeric(long_df["Value"], errors="coerce")
    
    # Drop rows where Year is NaN (in case non-year columns were melted)
    long_df = long_df.dropna(subset=['Year'])
    long_df["Year"] = long_df["Year"].astype(int)
    
    return long_df

che_long  = melt_indicator(df_che)
wr_long   = melt_indicator(df_wr)
wu_long   = melt_indicator(df_wu)
sr_long   = melt_indicator(df_sr)
su_long   = melt_indicator(df_su)
gem_long  = melt_indicator(df_gem)
pop_long  = melt_indicator(df_pop)
ren_long  = melt_indicator(df_ren)

all_long = pd.concat(
    [che_long, wr_long, wu_long, sr_long, su_long, gem_long, pop_long, ren_long],
    ignore_index=True
)

print("all_long shape:", all_long.shape)
all_long.head()


NameError: name 'df_che' is not defined

## STEP 5 — Build countries_df, indicators_df, values_df

These are the **3 tables** that will be inserted into MySQL:
- `countries_df` — unique list of countries  
- `indicators_df` — unique list of indicators  
- `values_df` — all actual data values  


In [ ]:
# 5.1 countries_df
countries_df = (
    all_long[["Country Code", "Country Name"]]
    .drop_duplicates()
    .sort_values("Country Code")
    .reset_index(drop=True)
)

countries_df["country_id"] = countries_df.index + 1
countries_df["region"] = None

countries_df = countries_df.rename(columns={
    "Country Code": "country_code",
    "Country Name": "country_name"
})

print("countries_df shape:", countries_df.shape)
countries_df.head()

# 5.2 indicators_df
indicators_df = (
    all_long[["Indicator Code", "Indicator Name"]]
    .drop_duplicates()
    .reset_index(drop=True)
)

indicators_df["indicator_id"] = indicators_df.index + 1

def extract_unit(name):
    m = re.search(r"\((.*?)\)", str(name))
    return m.group(1) if m else "original units"

indicators_df["unit"] = indicators_df["Indicator Name"].apply(extract_unit)

indicators_df = indicators_df.rename(columns={
    "Indicator Code": "indicator_code",
    "Indicator Name": "indicator_name"
})

print("indicators_df shape:", indicators_df.shape)
indicators_df.head()

# 5.3 values_df
country_code_to_id   = dict(zip(countries_df["country_code"],  countries_df["country_id"]))
indicator_code_to_id = dict(zip(indicators_df["indicator_code"], indicators_df["indicator_id"]))

values_df = all_long.copy()
values_df["country_id"]   = values_df["Country Code"].map(country_code_to_id)
values_df["indicator_id"] = values_df["Indicator Code"].map(indicator_code_to_id)

values_df = values_df.rename(columns={"Year": "year", "Value": "value"})

print("values_df shape:", values_df.shape)
values_df.head()


## STEP 6 — Connect to MySQL and create database


In [ ]:
# Conect to MySQL 
conn = pymysql.connect(
    host="127.0.0.1",
    user="root",
    password="student",   # ← your real password
    autocommit=True
)

cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS bigdata_project;")
cursor.execute("USE bigdata_project;")

print("Connected to MySQL and using database bigdata_project.")


## STEP 7 — Reset tables and recreate schema
We drop:
- indicator_values  
- indicators  
- countries  

Then recreate all three.


In [ ]:
cursor.execute("SET FOREIGN_KEY_CHECKS = 0;")
cursor.execute("DROP TABLE IF EXISTS indicator_values;")
cursor.execute("DROP TABLE IF EXISTS indicators;")
cursor.execute("DROP TABLE IF EXISTS countries;")
cursor.execute("SET FOREIGN_KEY_CHECKS = 1;")

cursor.execute("""
CREATE TABLE countries (
    country_id INT PRIMARY KEY,
    country_code VARCHAR(5),
    country_name VARCHAR(255),
    region VARCHAR(100)
);
""")

cursor.execute("""
CREATE TABLE indicators (
    indicator_id INT PRIMARY KEY,
    indicator_code VARCHAR(255),
    indicator_name VARCHAR(500),
    unit VARCHAR(100)
);
""")

cursor.execute("""
CREATE TABLE indicator_values (
    value_id BIGINT AUTO_INCREMENT PRIMARY KEY,
    country_id INT,
    indicator_id INT,
    year INT,
    value DOUBLE,
    FOREIGN KEY (country_id) REFERENCES countries(country_id),
    FOREIGN KEY (indicator_id) REFERENCES indicators(indicator_id)
);
""")

print("Tables countries, indicators, indicator_values created.")


## STEP 8 — Insert countries and indicators


In [ ]:
insert_countries_sql = """
    INSERT INTO countries (country_id, country_code, country_name, region)
    VALUES (%s, %s, %s, %s);
"""

for _, row in countries_df.iterrows():
    cursor.execute(insert_countries_sql, (int(row.country_id), row["country_code"], row["country_name"], row["region"]))

print("Inserted countries:", len(countries_df))

print("\n📤 Inserting indicators...")

insert_indicators_sql = """
    INSERT INTO indicators (indicator_id, indicator_code, indicator_name, unit)
    VALUES (%s, %s, %s, %s);
"""

for _, row in indicators_df.iterrows():
    cursor.execute(insert_indicators_sql, (int(row.indicator_id), row["indicator_code"], row["indicator_name"], row["unit"]))

print("Inserted indicators:", len(indicators_df))


## STEP 9 — Insert ~100,000 indicator_values in batches


In [ ]:
cursor.execute("TRUNCATE TABLE indicator_values;")

rows = []
for row in values_df.itertuples(index=False):
    val = None if pd.isna(row.value) else row.value
    rows.append((int(row.country_id), int(row.indicator_id), int(row.year), val))

total = len(rows)
print("Total rows to insert into indicator_values:", total)

insert_values_sql = """
    INSERT INTO indicator_values (country_id, indicator_id, year, value)
    VALUES (%s, %s, %s, %s);
"""

conn.autocommit(False)
batch_size = 5000
inserted = 0

for start in range(0, total, batch_size):
    batch = rows[start:start + batch_size]
    cursor.executemany(insert_values_sql, batch)
    conn.commit()
    inserted += len(batch)
    print(f"Inserted {inserted} / {total} rows...", end="\r")

conn.autocommit(True)
print(f"\nFinished inserting {inserted} rows into indicator_values.")


## STEP 10 — Create view *all_data*

This view joins all three tables into a single logical dataset that you can query directly.


In [ ]:
cursor.execute("""
CREATE OR REPLACE VIEW all_data AS
SELECT
    iv.value_id,
    iv.year,
    iv.value,
    c.country_id,
    c.country_code,
    c.country_name,
    c.region,
    i.indicator_id,
    i.indicator_code,
    i.indicator_name,
    i.unit
FROM indicator_values iv
JOIN countries  c ON iv.country_id   = c.country_id
JOIN indicators i ON iv.indicator_id = i.indicator_id;
""")

print("View all_data created.")


## STEP 11 — Sanity checks
We count rows and preview the joined dataset.


In [ ]:
print("\n Sanity checks:")

cursor.execute("SELECT COUNT(*) FROM countries;")
print("countries rows:", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM indicators;")
print("indicators rows:", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM indicator_values;")
print("indicator_values rows:", cursor.fetchone()[0])

sample_df = pd.read_sql("SELECT * FROM all_data LIMIT 10;", conn)
display(sample_df)

print("\n Database setup completed successfully.")


## Step 12 - Introducing regions back to the DB

Using the resources from https://www.worldbank.org/ext/en/where-we-work we were able to re-introduce regionality as dimension in our database. The accompanying region list excel file was used to intergate the regions back to our mySQL database. "Countries" such as Arab World or Europe where labeled as Regions, while ones such as Low Income Post-Demographic Divident where labeled as Socioeconomic Groups. Both of those labels can be utilized as reference and comparison points in the analysis.

In [ ]:
""""
REGION_FILE = "region_list.csv"

# 1. LOAD THE CORRECTED MAPPING FILE
try:
    # Load the CSV. It now contains the Country Code and the new Region.
    df_regions = pd.read_csv(REGION_FILE)
    
    # Clean the code/region values to ensure no trailing spaces cause SQL errors
    df_regions['country_code'] = df_regions['country_code'].astype(str).str.strip()
    df_regions['region'] = df_regions['region'].astype(str).str.strip()

    # 2. FILTER OUT AGGREGATES
    # We only update rows where a region was actually assigned (not 'NULL/IGNORE' or 'REGION')
    df_map_ready = df_regions
    
    print(f"Loaded mapping for {len(df_map_ready)} valid rows.")
    
except Exception as e:
    print(f"ERROR: Could not load {REGION_FILE}. Please check the filename.")
    raise e

# 3. CONNECT AND EXECUTE UPDATES
conn = pymysql.connect(
    host="127.0.0.1",
    user="root",
    password="student",
    database="project6001-db",
    autocommit=True
)
cursor = conn.cursor()

try:
    print("Executing final database update...")
    
    update_count = 0
    # Loop through the cleaned DataFrame and run the SQL UPDATE for each country
    for index, row in df_map_ready.iterrows():
        sql = "UPDATE countries SET region = %s WHERE country_id = %s"
        
        # Use the 'region' column as the value to SET, and 'country_code' for the WHERE clause
        cursor.execute(sql, (row['region'], row['country_code']))
        update_count += 1
        
    conn.commit()

    # 4. VERIFICATION
    cursor.execute("SELECT region, COUNT(*) FROM countries WHERE region IS NOT NULL GROUP BY region")
    
    print(f"\nSUCCESS! Updated {update_count} country records.")
    
    # Display final region counts
    df_verification = pd.DataFrame(cursor.fetchall(), columns=['Region', 'Count'])
    print("\nFinal Region Distribution in DB:")
    print(df_verification)

finally:
    conn.close()

""